In [1]:
import pandas as pd, numpy as np
from tqdm import tqdm
column_names1 = ['system', 'file', 'score', 'unknown', 'annotator']
column_names2 = ['file', 'score']

train = pd.read_csv('main-data/sets/TRAINSET', header=None, names=column_names1)
val = pd.read_csv('main-data/sets/DEVSET', header=None, names=column_names1)
train_mos_list = pd.read_csv('main-data/sets/train_mos_list.txt', header=None, names=column_names2)
val_mos_list = pd.read_csv('main-data/sets/val_mos_list.txt', header=None, names=column_names2)

ood_train = pd.read_csv('ood-data/sets/TRAINSET', header=None, names=column_names1)
ood_val = pd.read_csv('ood-data/sets/DEVSET', header=None, names=column_names1)
ood_train_mos_list = pd.read_csv('ood-data/sets/train_mos_list.txt', header=None, names=column_names2)
ood_unlabeled_mos_list = pd.read_csv('ood-data/sets/unlabeled_mos_list.txt', header=None, names=['file'])
ood_val_mos_list = pd.read_csv('ood-data/sets/val_mos_list.txt', header=None, names=column_names2)

In [2]:
for i, (file, score) in tqdm(train_mos_list.iterrows()):
    assert np.isclose(train[train.file == file].score.to_numpy().mean(), score)
for i, (file, score) in tqdm(val_mos_list.iterrows()):
    assert np.isclose(val[val.file == file].score.to_numpy().mean(), score)

4974it [00:13, 368.50it/s]
1066it [00:01, 1020.10it/s]


In [3]:
for i, (file, score) in tqdm(ood_train_mos_list.iterrows()):
    assert np.isclose(ood_train[ood_train.file == file].score.to_numpy().mean(), score)
for i, (file, score) in tqdm(ood_val_mos_list.iterrows()):
    assert np.isclose(ood_val[ood_val.file == file].score.to_numpy().mean(), score)

136it [00:00, 1462.30it/s]
136it [00:00, 1511.18it/s]


In [4]:
del train['unknown'], val['unknown'], ood_train['unknown'], ood_val['unknown']

In [5]:
train['subset'] = 'train'
val['subset'] = 'val'
ood_train['subset'] = 'ood_train'
ood_val['subset'] = 'ood_val'
ood_unlabeled_mos_list['subset'] = 'ood_unlabeled'

In [6]:
import os
train['file_exists'] = [os.path.isfile('main-data/wav/' + f) for f in train.file]
val['file_exists'] = [os.path.isfile('main-data/wav/' + f) for f in val.file]
ood_train['file_exists'] = [os.path.isfile('ood-data/wav/' + f) for f in ood_train.file]
ood_val['file_exists'] = [os.path.isfile('ood-data/wav/' + f) for f in ood_val.file]
ood_unlabeled_mos_list['file_exists'] = [os.path.isfile('ood-data/wav/' + f) for f in ood_unlabeled_mos_list.file]

In [7]:
df = pd.concat([train, val, ood_train, ood_val, ood_unlabeled_mos_list])
df

,system,file,score,annotator,subset,file_exists
0,sys64e2f,sys64e2f-utt9c183cd.wav,4.0,{}_30-39_bZPQE7w4Zl3g_Female_Valid_1_No,train,True
1,sys64e2f,sys64e2f-utt9c183cd.wav,4.0,{}_60-69_GoAQj1WzjXva_Female_Valid_1_No,train,True
2,sys64e2f,sys64e2f-utt9c183cd.wav,5.0,{}_18-29_A7NLzvbKwLkr_Male_Valid_1_No,train,True
3,sys64e2f,sys64e2f-utt9c183cd.wav,5.0,{}_30-39_7dgQ75qNKl4Z_Male_Valid_1_No,train,True
4,sys64e2f,sys64e2f-utt9c183cd.wav,5.0,{}_30-39_8m5l1ae4kl1j_Male_Valid_1_No,train,True
...,...,...,...,...,...,...
535,NaN,sys8569c-utt645eaf6.wav,NaN,NaN,ood_unlabeled,True
536,NaN,sys1b29a-uttdce3d1d.wav,NaN,NaN,ood_unlabeled,True
537,NaN,sys9900c-utt3509a4e.wav,NaN,NaN,ood_unlabeled,True
538,NaN,sysc0232-uttf369a3b.wav,NaN,NaN,ood_unlabeled,True


In [8]:
for i, row in tqdm(df.iterrows()):
    if type(row.system) == str:
        assert row.system == row.file.split('-')[0]

52527it [00:03, 17473.95it/s]


In [9]:
df.system = df.file.str.split('-').str[0]

In [10]:
df['utterance'] = df.file.str.split('-').str[1].str.split('.').str[0]

In [11]:
df = df[['subset', 'system', 'utterance', 'file', 'file_exists', 'score', 'annotator']]
df

,subset,system,utterance,file,file_exists,score,annotator
0,train,sys64e2f,utt9c183cd,sys64e2f-utt9c183cd.wav,True,4.0,{}_30-39_bZPQE7w4Zl3g_Female_Valid_1_No
1,train,sys64e2f,utt9c183cd,sys64e2f-utt9c183cd.wav,True,4.0,{}_60-69_GoAQj1WzjXva_Female_Valid_1_No
2,train,sys64e2f,utt9c183cd,sys64e2f-utt9c183cd.wav,True,5.0,{}_18-29_A7NLzvbKwLkr_Male_Valid_1_No
3,train,sys64e2f,utt9c183cd,sys64e2f-utt9c183cd.wav,True,5.0,{}_30-39_7dgQ75qNKl4Z_Male_Valid_1_No
4,train,sys64e2f,utt9c183cd,sys64e2f-utt9c183cd.wav,True,5.0,{}_30-39_8m5l1ae4kl1j_Male_Valid_1_No
...,...,...,...,...,...,...,...
535,ood_unlabeled,sys8569c,utt645eaf6,sys8569c-utt645eaf6.wav,True,NaN,NaN
536,ood_unlabeled,sys1b29a,uttdce3d1d,sys1b29a-uttdce3d1d.wav,True,NaN,NaN
537,ood_unlabeled,sys9900c,utt3509a4e,sys9900c-utt3509a4e.wav,True,NaN,NaN
538,ood_unlabeled,sysc0232,uttf369a3b,sysc0232-uttf369a3b.wav,True,NaN,NaN


In [12]:
df.to_csv('data_with_annotators.csv', index=False)

In [13]:
groupby = df.groupby(['subset', 'system', 'utterance', 'file', 'file_exists'])
df2 = pd.DataFrame({
    'score_mean': groupby.score.mean(),
    'score_std': groupby.score.std(),
    'n_scores': groupby.score.count()
}).reset_index()
df2

,subset,system,utterance,file,file_exists,score_mean,score_std,n_scores
0,ood_train,sys0c3c7,utt330976a,sys0c3c7-utt330976a.wav,True,1.230769,0.438529,13
1,ood_train,sys0c3c7,uttb54c043,sys0c3c7-uttb54c043.wav,True,1.545455,1.035725,11
2,ood_train,sys0c3c7,uttc349635,sys0c3c7-uttc349635.wav,True,1.428571,0.851631,14
3,ood_train,sys0c3c7,uttcde36fc,sys0c3c7-uttcde36fc.wav,True,1.000000,0.000000,13
4,ood_train,sys1b29a,utt014d122,sys1b29a-utt014d122.wav,True,3.066667,0.961150,15
...,...,...,...,...,...,...,...,...
6847,val,sysff05c,utt6f934c7,sysff05c-utt6f934c7.wav,True,2.875000,0.640870,8
6848,val,sysff05c,utt93dd6fa,sysff05c-utt93dd6fa.wav,True,3.375000,0.916125,8
6849,val,sysff05c,uttb21e76f,sysff05c-uttb21e76f.wav,True,3.125000,0.834523,8
6850,val,sysff05c,uttceb09ce,sysff05c-uttceb09ce.wav,True,2.375000,0.744024,8


In [14]:
df.subset.value_counts()

train            39792
val               8528
ood_train         1848
ood_val           1819
ood_unlabeled      540
Name: subset, dtype: int64

In [15]:
df2.subset.value_counts()

train            4974
val              1066
ood_unlabeled     540
ood_train         136
ood_val           136
Name: subset, dtype: int64

In [16]:
df2.to_csv('data.csv', index=False, float_format='%.4f')